In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import os

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

### Acquire Teams

In [2]:
teams = pd.read_csv('teams.csv')
teams

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends
5,0,1610612743,1976,2019,DEN,Nuggets,1976,Denver,Pepsi Center,19099.0,Stan Kroenke,Tim Connelly,Michael Malone,No Affiliate
6,0,1610612745,1967,2019,HOU,Rockets,1967,Houston,Toyota Center,18104.0,Tilman Fertitta,Daryl Morey,Mike D'Antoni,Rio Grande Valley Vipers
7,0,1610612746,1970,2019,LAC,Clippers,1970,Los Angeles,Staples Center,19060.0,Steve Ballmer,Michael Winger,Doc Rivers,Agua Caliente Clippers of Ontario
8,0,1610612747,1948,2019,LAL,Lakers,1948,Los Angeles,Staples Center,19060.0,Jerry Buss Family Trust,Rob Pelinka,Frank Vogel,South Bay Lakers
9,0,1610612748,1988,2019,MIA,Heat,1988,Miami,AmericanAirlines Arena,19600.0,Micky Arison,Pat Riley,Erik Spoelstra,Sioux Falls Skyforce


### Acquire Stats for Each Game

In [3]:
games = pd.read_csv('games.csv')
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-05-26,42000102,Final,1610612755,1610612764,2020,1610612755,120.0,0.557,0.684,...,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1
1,2021-05-26,42000132,Final,1610612752,1610612737,2020,1610612752,101.0,0.383,0.739,...,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1
2,2021-05-26,42000142,Final,1610612762,1610612763,2020,1610612762,141.0,0.544,0.774,...,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1
3,2021-05-25,42000112,Final,1610612751,1610612738,2020,1610612751,130.0,0.523,0.955,...,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1
4,2021-05-25,42000152,Final,1610612756,1610612747,2020,1610612756,102.0,0.465,0.933,...,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0


### Cleaning Data

In [4]:
# Dropping HOME_TEAM_ID and VISITOR_TEAM_ID since their numbers are the same as
# TEAM_ID_home and TEAM_ID_away
games.drop(columns = ['HOME_TEAM_ID','VISITOR_TEAM_ID'], inplace=True)

In [5]:
# Changing Game_Date_EST to datetime
games['GAME_DATE_EST']= pd.to_datetime(games['GAME_DATE_EST'])
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24677 entries, 0 to 24676
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   GAME_DATE_EST     24677 non-null  datetime64[ns]
 1   GAME_ID           24677 non-null  int64         
 2   GAME_STATUS_TEXT  24677 non-null  object        
 3   SEASON            24677 non-null  int64         
 4   TEAM_ID_home      24677 non-null  int64         
 5   PTS_home          24578 non-null  float64       
 6   FG_PCT_home       24578 non-null  float64       
 7   FT_PCT_home       24578 non-null  float64       
 8   FG3_PCT_home      24578 non-null  float64       
 9   AST_home          24578 non-null  float64       
 10  REB_home          24578 non-null  float64       
 11  TEAM_ID_away      24677 non-null  int64         
 12  PTS_away          24578 non-null  float64       
 13  FG_PCT_away       24578 non-null  float64       
 14  FT_PCT_away       2457

In [6]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-05-26,42000102,Final,2020,1610612755,120.0,0.557,0.684,0.429,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1
1,2021-05-26,42000132,Final,2020,1610612752,101.0,0.383,0.739,0.364,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1
2,2021-05-26,42000142,Final,2020,1610612762,141.0,0.544,0.774,0.487,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1
3,2021-05-25,42000112,Final,2020,1610612751,130.0,0.523,0.955,0.447,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1
4,2021-05-25,42000152,Final,2020,1610612756,102.0,0.465,0.933,0.308,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0


In [7]:
# We can drop this column
games.GAME_STATUS_TEXT.value_counts()

Final    24677
Name: GAME_STATUS_TEXT, dtype: int64

In [8]:
games.drop(columns = 'GAME_STATUS_TEXT', inplace=True)
games.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home',
       'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'TEAM_ID_away', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away',
       'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [9]:
# checking for nulls
games.isnull().any()

GAME_DATE_EST     False
GAME_ID           False
SEASON            False
TEAM_ID_home      False
PTS_home           True
FG_PCT_home        True
FT_PCT_home        True
FG3_PCT_home       True
AST_home           True
REB_home           True
TEAM_ID_away      False
PTS_away           True
FG_PCT_away        True
FT_PCT_away        True
FG3_PCT_away       True
AST_away           True
REB_away           True
HOME_TEAM_WINS    False
dtype: bool

In [10]:
# This tells us the null values for these games are in October 
# There are only 99 rows out of 24,677...I think it's okay to drop

games[games.PTS_home.isnull()==True].GAME_DATE_EST.value_counts()

2003-10-14    10
2003-10-17     9
2003-10-11     9
2003-10-24     9
2003-10-22     8
2003-10-18     7
2003-10-16     6
2003-10-23     6
2003-10-21     5
2003-10-19     5
2003-10-10     5
2003-10-15     4
2003-10-20     4
2003-10-09     4
2003-10-12     3
2003-10-08     2
2003-10-13     2
2003-10-07     1
Name: GAME_DATE_EST, dtype: int64

In [12]:
games.drop(games[games.PTS_home.isnull()==True].index, inplace = True)
games.shape

(24578, 18)

In [13]:
# Now we don't have any nulls!
games.isnull().any()

GAME_DATE_EST     False
GAME_ID           False
SEASON            False
TEAM_ID_home      False
PTS_home          False
FG_PCT_home       False
FT_PCT_home       False
FG3_PCT_home      False
AST_home          False
REB_home          False
TEAM_ID_away      False
PTS_away          False
FG_PCT_away       False
FT_PCT_away       False
FG3_PCT_away      False
AST_away          False
REB_away          False
HOME_TEAM_WINS    False
dtype: bool

In [14]:
# Creating the total points column, this will be our target
games['total_points'] = games.PTS_home + games.PTS_away

In [15]:
games.head()

,GAME_DATE_EST,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,total_points
0,2021-05-26,42000102,2020,1610612755,120.0,0.557,0.684,0.429,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1,215.0
1,2021-05-26,42000132,2020,1610612752,101.0,0.383,0.739,0.364,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1,193.0
2,2021-05-26,42000142,2020,1610612762,141.0,0.544,0.774,0.487,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1,270.0
3,2021-05-25,42000112,2020,1610612751,130.0,0.523,0.955,0.447,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1,238.0
4,2021-05-25,42000152,2020,1610612756,102.0,0.465,0.933,0.308,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0,211.0


In [20]:
games.SEASON.value_counts().sort_index()

2003    1286
2004    1362
2005    1432
2006    1419
2007    1411
2008    1425
2009    1424
2010    1422
2011    1104
2012    1420
2013    1427
2014    1418
2015    1416
2016    1405
2017    1382
2018    1378
2019    1241
2020    1206
Name: SEASON, dtype: int64